# PTT 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 印出最新文章的「作者」「標題」「時間」
* ② 印出第一頁所有文章的「作者」「標題」「時間」


### ① 印出最新文章的「作者」「標題」「時間」

In [194]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [195]:
url='https://www.ptt.cc/bbs/NBA/index.html' #看板任何一頁的 url都行
r=requests.get(url)
res=BeautifulSoup(r.text)

In [196]:
newtag=res.find('a',string='最新') #根據string找到標籤
newlink='https://www.ptt.cc/'+newtag['href'] #獲得最新文章頁面的超連結後，記得組成完整網址

#獲取最新文章所在頁面的原始碼 
res2=BeautifulSoup(requests.get(newlink).text)

In [199]:
#將時間整數轉成日期
def TransTime(n):
        timeint=int(n) #字串轉成整數資料，localtime只接受整數
        times=time.localtime(timeint) #轉換日期
        timeresult=time.strftime('%Y-%m-%d %H:%M:%S',times) #格式化
        return timeresult

In [200]:
#找出最新文章
tags=res2.find_all('div',class_="r-ent")

titlelist=[]
authorlist=[]
timelist=[]
for tag in tags:
    try:
        timedata=tag.a['href'].split('.')[1] #從標籤 a的屬性 href取出時間整數值(資料型態:字串)
        returntime=TransTime(timedata) #從函式回傳一個轉換後的時間值
        timelist.append(returntime) #將此回傳值加到時間列表
        authortag=tag.find('div',class_="author") #找作者標籤
        authorlist.append(authortag.text) #建立作者list
        titlelist.append(tag.a.string) #建立文章標題list
        
    except:
        continue

#建立dataframe
df=pd.DataFrame({'時間':timelist,'作者':authorlist,'標題':titlelist})
print('最新文章時間',df['時間'].max()) #找出最新時間
n=0
for i in df.iloc[:,0]: #對時間欄內所有時間比對
    n+=1
    if i ==df['時間'].max(): #若等於最新時間，輸出那一行(時間 &作者 &標題)
        print(df.iloc[n-1,:])


最新文章時間 2019-12-26 15:15:24
時間        2019-12-26 15:15:24
作者                  fukawa947
標題    [新聞] 看見季後賽曙光　保羅心境轉變想為雷霆
Name: 6, dtype: object


### ② 印出第一頁所有文章的「作者」「標題」「時間」

In [237]:
#由看板任一頁url獲得最新一頁url
def newestPage(url):
    r=requests.get(url)
    res=BeautifulSoup(r.text)
    newtag=res.find('a',string='最新') #根據string找到標籤
    newlink='https://www.ptt.cc/'+newtag['href'] #獲得最新文章頁面的超連結後，記得組成完整網址
    return newlink
    
newlink=newestPage('https://www.ptt.cc/bbs/NBA/index6509.html') #任一頁url丟入函式，獲得最新頁url

In [250]:
#建立函式:從最新一頁逐漸往上一頁搜尋文章
def PageData(link):
    r=BeautifulSoup(requests.get(link).text) #解析最新頁htnl網頁碼
    #搜尋標籤   
    rtags=r.find_all('div',class_="r-ent")
    titlelist=[]
    authorlist=[]
    timelist=[]

    for rtag in rtags:
        try:
            titlelist.append(rtag.a.text) #建立文章標題list
            authortag=rtag.find('div',class_="author") #找作者標籤
            authorlist.append(authortag.text) #建立作者list
            timedata=rtag.a['href'].split('.')[1] #從標籤 a的屬性 href取出時間整數值(資料型態:字串)
            returntime=TransTime(timedata) #從函式回傳一個轉換後的時間值
            timelist.append(returntime) #將此回傳值加到時間列表

        except:
            continue

    #建立dataframe印出文章資料
    print(pd.DataFrame({'作者':authorlist,'時間':timelist,'標題':titlelist}))
    
    #回傳上一頁url
    newtag=r.find('a',string='‹ 上頁')
    lastpagelink='https://www.ptt.cc/'+newtag['href'] #獲得上一頁頁面的超連結後，記得組成完整網址
    return lastpagelink

#呼叫函式印出最新一頁內容
PageData(newlink)

              作者                   時間                                      標題
0      jimmy5680  2019-12-26 13:34:07              [新聞] 雷納德19投11中轟35+12賞湖人4連敗
1       james008  2019-12-26 13:52:04                [花邊] 詹姆斯：開場被貝弗利頂到腹股溝後，感覺
2          Rambo  2019-12-26 13:58:14      [BOX ] Pelicans 112:100 Nuggets 數據
3     satelliter  2019-12-26 14:02:58                      [外絮] 癌末青年實現與老詹握手願望
4        pautaid  2019-12-26 14:42:20                [新聞] 詹姆斯飆追平三分球  挨瘋狗貝再見火鍋
5     currykukuo  2019-12-26 14:49:54                  [新聞] NBA／杰倫出訪　拿走加國的第一次
6      fukawa947  2019-12-26 15:15:24                 [新聞] 看見季後賽曙光　保羅心境轉變想為雷霆
7          lifuz  2019-12-26 15:25:36                [花邊] 影／救人喔！摔進觀眾席…一眉重壓好萊塢
8     currykukuo  2019-12-26 15:28:24                [外絮] PG:西決基本上已經確定是湖人跟快艇打
9      bbbyes123  2019-12-26 15:55:29                [情報] 名記:詹姆斯瘸腿走出更衣室，稱被撞傷後
10     Pinka5566  2019-12-26 16:16:17               Re: [情報] WoJ:Kawhi肌腱炎帶傷上陣
11        pneumo  2019-12-26 16:35:36             [情報] Beverely講

'https://www.ptt.cc//bbs/NBA/index6511.html'

In [256]:
#使用迴圈可抓取多頁
count=0
link=newlink
while count<2: #想印出幾頁就跑幾次迴圈
    link=PageData(link) #將回傳的上一頁url做為新的url丟進函式獲得另一頁的資料
    count+=1

              作者                   時間                                      標題
0      jimmy5680  2019-12-26 13:34:07              [新聞] 雷納德19投11中轟35+12賞湖人4連敗
1       james008  2019-12-26 13:52:04                [花邊] 詹姆斯：開場被貝弗利頂到腹股溝後，感覺
2          Rambo  2019-12-26 13:58:14      [BOX ] Pelicans 112:100 Nuggets 數據
3     satelliter  2019-12-26 14:02:58                      [外絮] 癌末青年實現與老詹握手願望
4        pautaid  2019-12-26 14:42:20                [新聞] 詹姆斯飆追平三分球  挨瘋狗貝再見火鍋
5     currykukuo  2019-12-26 14:49:54                  [新聞] NBA／杰倫出訪　拿走加國的第一次
6      fukawa947  2019-12-26 15:15:24                 [新聞] 看見季後賽曙光　保羅心境轉變想為雷霆
7          lifuz  2019-12-26 15:25:36                [花邊] 影／救人喔！摔進觀眾席…一眉重壓好萊塢
8     currykukuo  2019-12-26 15:28:24                [外絮] PG:西決基本上已經確定是湖人跟快艇打
9      bbbyes123  2019-12-26 15:55:29                [情報] 名記:詹姆斯瘸腿走出更衣室，稱被撞傷後
10     Pinka5566  2019-12-26 16:16:17               Re: [情報] WoJ:Kawhi肌腱炎帶傷上陣
11        pneumo  2019-12-26 16:35:36             [情報] Beverely講

### ③ 試著爬爬看其他版的文章

In [258]:
#ptt food
count=0
link='https://www.ptt.cc/bbs/Food/index.html'
while count<2:
    link=PageData(link) 
    count+=1

            作者                   時間                            標題
0       AlphaD  2019-12-26 04:36:32          [食記] 日本大阪 螃蟹道樂 道頓堀本店
1    chrisxmen  2019-12-26 07:50:20            Fw: [食記] 指宿 唐船峽流水麵
2     MarkFung  2019-12-26 08:36:51            [食記] 基隆市七堵區 七堵臭粿仔湯
3    leafphoto  2019-12-26 11:13:32                  [食記] 高雄 阿強碳烤
4   tiantian27  2019-12-26 11:50:49  [食記] 台北 LadyM 遠百信義A13！挑高空間太美
5         lkjt  2019-12-26 13:14:01  [食記] 台北 CHAMBISTRO 享．香檳海鮮餐酒館
6      ally701  2019-12-26 15:45:57      [食記] 台北松山 民生社區 玉小蘭 漢堡超好吃
7      ally701  2019-12-26 15:48:11     [食記] 台北西門町 必可蜜飲料 綠寶珍珠奶茶大推
8      designW  2019-12-26 16:28:56              [食記] 高雄路竹-阿美土魠魚羹
9      Dilbert  2012-12-16 23:59:42           [公告] Food板 板規 V3.91
10    bluefish  2007-09-28 09:53:46     [公告] 發文請在標題加上地區及提供地址電話。^^
11     Dilbert  2005-10-01 10:11:06        [公告] 文章被刪除者請洽精華區的資源回收桶
12     Dilbert  2017-06-04 07:27:49        [公告] 新增板規22：發文禁附延伸閱讀連結
            作者                   時間                                  標題
0   